In [2]:
import numpy as np
np.random.seed(17)

from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

In [8]:
# define 
docs = ['king is man',
'a king marrys queen',
'a queen marrys king',
'some unrelated workds'
'queen is woman']
docs

['king is man',
 'a king marrys queen',
 'a queen marrys king',
 'some unrelated workdsqueen is woman']

In [9]:
vocab_size = 10

In [15]:
oh = np.array(one_hot('king queen man woman unrelated', vocab_size)).reshape(-1,1)

In [16]:
oh

array([[9],
       [8],
       [1],
       [5],
       [6]])

In [14]:
enc = OneHotEncoder()
enc.fit(np.array(range(50)).reshape(-1,1))
oh_enc = enc.transform(oh).toarray()
oh_enc

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
     

In [17]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]
encoded_docs

[[9, 8, 1], [7, 9, 3, 8], [7, 8, 3, 9], [7, 6, 5, 8, 5]]

In [18]:
max_length = 24
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs

array([[9, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [7, 9, 3, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [7, 8, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [7, 6, 5, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]], dtype=int32)

In [20]:
tuples = np.empty((0,2))
for padded_doc in padded_docs:
    length = len(padded_doc)
    for i in range(length):
        if padded_doc[i] != 0:
            if i < length - 1 & padded_doc[i+1] != 0:
                tuples = np.append(tuples, [[padded_doc[i], padded_doc[i+1]]], axis=0)
                if i < length -2 & padded_doc[i+2] != 0:
                    tuples = np.append(tuples, [[padded_doc[i], padded_doc[i+2]]], axis=0)
            if i > 0:
                tuples = np.append(tuples, [[padded_doc[i] , padded_doc[i-1]]], axis=0)
                if i > 1:
                    tuples = np.append(tuples, [[padded_doc[i], padded_doc[i-2]]], axis=0)
                    
print(tuples.shape)
tuples

(28, 2)


array([[ 8.,  9.],
       [ 1.,  8.],
       [ 1.,  9.],
       [ 7.,  9.],
       [ 7.,  3.],
       [ 9.,  3.],
       [ 9.,  7.],
       [ 3.,  9.],
       [ 3.,  7.],
       [ 8.,  3.],
       [ 8.,  9.],
       [ 8.,  3.],
       [ 8.,  7.],
       [ 3.,  8.],
       [ 3.,  7.],
       [ 9.,  3.],
       [ 9.,  8.],
       [ 7.,  6.],
       [ 7.,  5.],
       [ 6.,  5.],
       [ 6.,  7.],
       [ 5.,  6.],
       [ 5.,  7.],
       [ 8.,  5.],
       [ 8.,  5.],
       [ 8.,  6.],
       [ 5.,  8.],
       [ 5.,  5.]])

In [21]:
onehotlabels_x = enc.transform(tuples[:,0].reshape(-1, 1)).toarray()

print(onehotlabels_x.shape)
onehotlabels_x

(28, 50)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
onehotlabel_y = enc.transform(tuples[:,1].reshape(-1,1)).toarray()

print(onehotlabel_y.shape)
onehotlabel_y

(28, 50)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
model = Sequential()

input = Dense(50, input_shape=(50,), activation='relu')
model.add(input)
bottleneck = Dense(2, activation='relu')
model.add(bottleneck)
model.add(Dense(50, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

model.fit(onehotlabels_x,onehotlabel_y,epochs=1,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 102       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                150       
Total params: 2,802
Trainable params: 2,802
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
28/28 [==============================] - 0s 11ms/step - loss: 0.0979 - acc: 0.9800


In [26]:
model = Sequential()

model.add(input)
model.add(bottleneck)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy',metrics=['acc'])
test = model.predict(oh_enc)
print(test)
print(test[0,:] - test[2,:] + test[3,:] - test[2,:])

[[ 0.          0.11610074]
 [ 0.05456454  0.04941866]
 [ 0.          0.06904231]
 [ 0.          0.02930483]
 [ 0.          0.1530893 ]]
[ 0.          0.00732094]
